In [1]:
!pip install langchain openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.3 MB/s eta 0:00:00


In [2]:
import os
os.environ['OPENAI_API_KEY'] = 'your api key'

In [6]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain

# first initialize the large language model
llm = OpenAI(
	temperature=0.3,

	model_name="text-davinci-003"
)

# now initialize the conversation chain
conversation = ConversationChain(llm=llm)

In [7]:
print(conversation.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [8]:
from langchain.chains.conversation.memory import ConversationBufferMemory
conversation_buf = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

In [9]:
conversation_buf("Can you please tell me your name AI?")


{'input': 'Can you please tell me your name AI?',
 'history': '',
 'response': " Sure! My name is AI-01. It's nice to meet you!"}

In [13]:
from langchain.callbacks import get_openai_callback

def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result


In [11]:
count_tokens(
    conversation_buf,
    "This book introduces the concept of a Big Hairy Audacious Goal (BHAG), a long-term, ambitious,\
     and inspiring goal that can motivate and align the efforts of employees and stakeholders."
)

Spent a total of 178 tokens


" That sounds really interesting! I'm not familiar with the concept of a Big Hairy Audacious Goal, but I'm sure it can be a great way to motivate and align people. Can you tell me more about it?"

In [14]:
count_tokens(
    conversation_buf,
    "The BHAG can help an organization focus on a long-term goal, create a sense of urgency, and attract and retain top talent. The book encourages readers to embrace their BHAGs and galvanize their organizations towards success."
)

Spent a total of 293 tokens


" That's really inspiring! It sounds like the BHAG can be a great tool for organizations to focus on their long-term goals and create a sense of urgency. I'm sure it can help attract and retain top talent as well. Do you have any other insights about the BHAG?"

In [15]:
count_tokens(
    conversation_buf,
    "What concept this book introduces?"
)

Spent a total of 346 tokens


' This book introduces the concept of a Big Hairy Audacious Goal (BHAG), a long-term, ambitious, and inspiring goal that can motivate and align the efforts of employees and stakeholders.'

In [16]:
print(conversation_buf.memory.buffer)

Human: Can you please tell me your name AI?
AI:  Sure! My name is AI-01. It's nice to meet you!
Human: This book introduces the concept of a Big Hairy Audacious Goal (BHAG), a long-term, ambitious,     and inspiring goal that can motivate and align the efforts of employees and stakeholders.
AI:  That sounds really interesting! I'm not familiar with the concept of a Big Hairy Audacious Goal, but I'm sure it can be a great way to motivate and align people. Can you tell me more about it?
Human: The BHAG can help an organization focus on a long-term goal, create a sense of urgency, and attract and retain top talent. The book encourages readers to embrace their BHAGs and galvanize their organizations towards success.
AI:  That's really inspiring! It sounds like the BHAG can be a great tool for organizations to focus on their long-term goals and create a sense of urgency. I'm sure it can help attract and retain top talent as well. Do you have any other insights about the BHAG?
Human: What co

In [17]:
from langchain.chains.conversation.memory import ConversationSummaryMemory

conversation_sum = ConversationChain(
	llm=llm,
	memory=ConversationSummaryMemory(llm=llm)
)


In [18]:
print(conversation_sum.memory.prompt.template)

Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:


In [19]:
# without count_tokens we'd call `conversation_sum("Good morning AI!")`
# but let's keep track of our tokens:
count_tokens(
    conversation_sum,
    "Can you please tell me your name AI?"
)

Spent a total of 319 tokens


" Sure! My name is AI-01. I'm a conversational AI created to help you with your questions."

In [20]:
count_tokens(
    conversation_sum,
    "This book introduces the concept of a Big Hairy Audacious Goal (BHAG), a long-term, ambitious,\
     and inspiring goal that can motivate and align the efforts of employees and stakeholders."
)

Spent a total of 592 tokens


" That sounds like an interesting concept. I'm AI-01, a conversational AI created to help with questions. I believe that artificial intelligence can be a force for good, and can help to achieve ambitious goals. It can help to automate processes, make decisions, and provide insights that can help to improve efficiency and accuracy."

In [21]:
count_tokens(
    conversation_sum,
    "The BHAG can help an organization focus on a long-term goal, create a sense of urgency, and attract and retain top talent. The book encourages readers to embrace their BHAGs and galvanize their organizations towards success."
)

Spent a total of 788 tokens


" Absolutely! BHAGs are an important part of any organization's strategy and can help to create a sense of purpose and direction. They can also help to motivate and inspire employees to work towards a common goal. Additionally, BHAGs can help to attract and retain top talent, as they provide an opportunity to work on something meaningful and challenging."

In [22]:
count_tokens(
    conversation_sum,
    "What concept this book introduces?"
)

Spent a total of 777 tokens


" I'm sorry, I don't know the answer to that question."

In [23]:
count_tokens(
    conversation_sum,
    "the book encourages readers of what?"
)

Spent a total of 821 tokens


" I'm sorry, I do not know the answer to that question."

In [24]:
print(conversation_sum.memory.buffer)


The human asks what the AI thinks of artificial intelligence. The AI introduces itself as AI-01, a conversational AI created to help with questions, and believes that artificial intelligence is a force for good that can help to achieve ambitious goals such as Big Hairy Audacious Goals (BHAGs). It can help to automate processes, make decisions, and provide insights that can help to improve efficiency and accuracy. The human then explains that BHAGs can help an organization focus on a long-term goal, create a sense of urgency, and attract and retain top talent. The AI agrees, noting that BHAGs are an important part of any organization's strategy and can help to create a sense of purpose and direction, motivate and inspire employees, and provide an opportunity to work on something meaningful and challenging. When asked what concept the book introduces, the AI responds that it does not know the answer.


In [25]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
#In this instance, we set k=1
conversation_bufw = ConversationChain(
	llm=llm,
	memory=ConversationBufferWindowMemory(k=1)
)

In [26]:
count_tokens(
    conversation_bufw,
    "Good morning AI!"
)

Spent a total of 85 tokens


" Good morning! It's a beautiful day today, isn't it? How can I help you?"

In [27]:
count_tokens(
    conversation_bufw,
    "I want to know more about Singularity, what it is and when it is coming."
)


Spent a total of 155 tokens


' Singularity is a term used to describe the moment when artificial intelligence surpasses human intelligence. It is believed that this moment will happen sometime in the next few decades. However, it is difficult to predict exactly when this will occur.'

In [28]:
count_tokens(
    conversation_bufw,
    "What I want to know?"
)

Spent a total of 198 tokens


' You asked about Singularity, what it is and when it is coming. Singularity is a term used to describe the moment when artificial intelligence surpasses human intelligence. It is believed that this moment will happen sometime in the next few decades. However, it is difficult to predict exactly when this will occur.'

In [29]:
count_tokens(
    conversation_bufw,
    "Give me the technology behind it"
)


Spent a total of 211 tokens


' The technology behind Singularity is artificial intelligence (AI). AI is a branch of computer science that focuses on creating intelligent machines that can think and act like humans. AI is used in many different areas, such as robotics, natural language processing, computer vision, and machine learning. AI is also used to create autonomous systems that can make decisions without human input.'

In [34]:
count_tokens(
    conversation_bufw,
    "What I want to know?"
)

Spent a total of 109 tokens


" I'm sorry, I'm not sure what you're asking. Could you please rephrase your question?"

In [31]:
from langchain.chains.conversation.memory import ConversationSummaryBufferMemory


In [35]:
conversation_sum_bufw = ConversationChain(
    llm=llm,
    memory=ConversationSummaryBufferMemory(
        llm=llm,
        max_token_limit=650
    ),
)

In [36]:
print(conversation_sum_bufw.memory.prompt.template)


Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:
